In [1]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from PIL import Image

In [55]:
imgPath = "/Volumes/Aortas/Aorta/preprocessedImages/cellSize/stitched/cropped1-8/nuclei"
segmPath = "/Volumes/Aortas/Aorta/segmentations/cellSize/stitchCyto"
saveName = "/Volumes/Aortas/Aorta/results/density/densityStitched.csv"

In [ ]:
#hand measured
areaPath = "/Users/jones/Downloads/areasStitched"

In [56]:
folders = [x[0] for x in os.walk(segmPath)]
folders = sorted(folders)

In [58]:
dfResultsAfterP7 = pd.DataFrame()
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f]
    if files:
        age = folder.split("/")[-2]
        if (age != "P03") and (age != "P05") and (age != "P07"):
            #for P3, P5, P7 too few results => hand measurements of area
            aorta = folder.split("/")[-1]
            imagePath = join(imgPath, age, aorta)
            fileImage = [f for f in listdir(imagePath) if isfile(join(imagePath, f)) if ".DS_Store" not in f if "._" not in f]
            im = Image.open(join(imagePath,fileImage[0]) )
            cropping_parameter_width = int(im.size[0]/550)
            cropping_parameter_height = int(im.size[1]/550)
            crop_width = im.size[0]/cropping_parameter_width
            crop_height = im.size[1]/cropping_parameter_height
            field = np.zeros((cropping_parameter_height, cropping_parameter_width))
            for i in range(cropping_parameter_height):
                for j in range(cropping_parameter_width):
                    if isfile(join(segmPath,age, aorta, "Cropped_IMG-" + str((i*cropping_parameter_width) + j + 1) + "_seg.npy")):
                        field[i,j] = 1
            for j in range(cropping_parameter_width):
                excludeUpper = False
                excludeLower = False
                i = 0
                while((excludeUpper == False) and (i<cropping_parameter_height)):
                    if(field[i,j] == 1):
                        field[i,j] = 0
                        excludeUpper = True
                    else:
                        i += 1
                i = cropping_parameter_height -1
                while((excludeLower == False) and (i>0)):
                    if(field[i,j] == 1):
                        field[i,j] = 0
                        excludeLower = True
                    else:
                        i -= 1
            for i in range(cropping_parameter_height):
                for j in range(cropping_parameter_width):
                    if field[i,j] == 1:
                        masks = np.load(join(segmPath, age, aorta, "Cropped_IMG-" + str((i*cropping_parameter_width) + j + 1) + "_seg.npy"), allow_pickle=True).item()['masks']
                        nuclei = len(np.unique(masks)) -1
                        density = nuclei/(len(masks)*len(masks[0]))
                        field[i,j] = density

            densities = []
            position = []
            imageNumber = []
            for i in range(cropping_parameter_height):
                for j in range(cropping_parameter_width):
                    if field[i,j] != 0:
                        densities.append(field[i,j])
                        position.append(j/(cropping_parameter_width-1))
                        imageNumber.append((i*cropping_parameter_width) + j + 1)

            dfResult = pd.DataFrame({"density": densities, "position": position, "imageNumber": imageNumber})
            dfResult["age"] = age
            dfResult["aorta"] = aorta
            os.chdir(savePath)
            #if not os.path.exists(join(savePath, age)):
                #os.mkdir(age)
            #os.chdir(join(savePath, age))
            #if not os.path.exists(join(savePath, age, aorta)):
                #os.mkdir(aorta)
            #os.chdir(join(savePath,age, aorta))
            #dfResult.to_csv((join(savePath,age, aorta,"density.csv")))
            dfResultsAfterP7 = pd.concat([dfResultsAfterP7, dfResult])


KeyboardInterrupt: 

In [ ]:
#hand measured areas

In [ ]:
folders = [x[0] for x in os.walk(areaPath)]
folders = sorted(folders)

In [65]:
dfCroppingParam = pd.read_csv("/Volumes/Aortas/Aorta/results/croppingParam_P03_P05_P07.csv")
dfResultsP3_5_7 = pd.DataFrame()
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".csv" in f if "._" not in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        cropping_parameter_width = dfCroppingParam.loc[(dfCroppingParam.age == age) & (dfCroppingParam.aorta == aorta)]["cropping_parameter_width"].values[0]
        cropping_parameter_height = dfCroppingParam.loc[(dfCroppingParam.age == age) & (dfCroppingParam.aorta == aorta)]["cropping_parameter_height"].values[0]
        field = np.zeros((cropping_parameter_height, cropping_parameter_width))
        dfArea = pd.read_csv(join(folder, file))

        densities = []
        position = []
        imageNumbers = []
        for index, row in dfArea.iterrows():
            area = row["Area"]
            imageNumber = int(row["Label"].split(".tif")[0].split("-")[-1])
            segmFile = row["Label"].split(".tif")[0] + "_seg.npy"
            masks = np.load(join(segmPath, age, aorta, segmFile), allow_pickle = True).item()['masks']
            numberNuclei = len(np.unique(masks)) - 1
            i = int((imageNumber-1)/cropping_parameter_width)
            j = int((imageNumber-1)%cropping_parameter_width)
            densities.append(numberNuclei/area)
            position.append(j/(cropping_parameter_width-1))
            imageNumbers.append(imageNumber)

        dfResult = pd.DataFrame({"density": densities, "position": position, "imageNumber": imageNumbers})
        dfResult["age"] = age
        dfResult["aorta"] = aorta
        dfResultsP3_5_7 = pd.concat([dfResultsP3_5_7, dfResult])

dfResultsP3_5_7["positionPlot"] = np.round(dfResultsP3_5_7["position"], decimals = 1)
dfResultsAll = pd.concat([dfResultsAfterP7, dfResultsP3_5_7], ignore_index = True)
dfResultsAll["densityMicro"] = dfResultsAll["density"] *1.162803946*1.162803946
dfResultsAll.to_csv(saveName)